In [1]:
import os
BASE_DIR = 'C:/Users/lbisa/Mi unidad (leonardo.bisaro@gmail.com)/Cripto/python/trd_app'
os.chdir(BASE_DIR)

import scripts.functions as fn
from scripts.indicators import supertrend
from datetime import datetime
import pandas as pd
import local__config as local
from binance.client import Client as BinanceClient



In [4]:
#Conexion con Binance sin API-KEY

client = BinanceClient()
def get_klines(symbol,interval,limit):
        
        klines = client.get_historical_klines(symbol=symbol, 
                                                   interval=interval,
                                                   limit=limit)
        df = pd.DataFrame(klines)
        df = df.iloc[:, :6]
        df.columns = ["datetime", "open", "high", "low", "close", "volume"]
        df['open'] = df['open'].astype('float')
        df['high'] = df['high'].astype('float')
        df['low'] = df['low'].astype('float')
        df['close'] = df['close'].astype('float')
        df['volume'] = df['volume'].astype('float')
        df['datetime'] = pd.to_datetime(df['datetime'], unit='ms') - pd.Timedelta('3 hr')
        return df

In [7]:

limit = 100

symbols = ['ETHUSDT','BTCUSDT','BNBUSDT']
intervals = '1h','4h','1d','3d'
for symbol in symbols:
    print(f'{symbol.rjust(12)}', end=' ')
    for interval in intervals:
        df = get_klines(symbol=symbol,interval=interval,limit=limit)
        df = supertrend(df)
        df['signal_buy'] = df[df['st_trigger']>0]['close']
        df['signal_sell'] = df[df['st_trigger']<0]['close']
        last = df.iloc[-1]
        trend = 'Neutro'
        if last['st_trend'] > 0:
            trend = 'Alcista' 
        elif last['st_trend'] < 0:
            trend = 'Bajista' 
        print(f'{interval.rjust(3)} {trend.ljust(10)}', end=' ')
    print(' ')

     ETHUSDT  1h Alcista     4h Alcista     1d Bajista     3d Alcista     
     BTCUSDT  1h Alcista     4h Bajista     1d Bajista     3d Alcista     
     BNBUSDT  1h Alcista     4h Bajista     1d Alcista     3d Alcista     


In [8]:
indicators = [{'col': 'st_low',
               'name': 'ST Low',
               'color': 'green'
               },
              {'col': 'st_high',
               'name': 'ST High',
               'color': 'red'
               },
               ]
events =     [{'df': df,
               'col': 'signal_buy',
               'name': 'BUY',
               'color': 'yellow',
               'symbol': 'triangle-up',
               },
              {'df': df,
               'col': 'signal_sell',
               'name': 'SELL',
               'color': 'yellow',
               'symbol': 'triangle-down',
               },
               ]

fig = fn.ohlc_chart(df, show_pnl = False, events=events, indicators=indicators)
fig.show()